### Indeed.ca scraper

In [ ]:
# https://medium.com/@msalmon00/web-scraping-job-postings-from-indeed-96bd588dcb4b
# referenced code in Mark Salmon's Medium article

In [1]:
import requests
import bs4
from bs4 import BeautifulSoup as soup

import pandas as pd
import time

In [2]:
url = "https://www.indeed.ca/jobs?q=data+intern&l=Toronto%2C+ON" # test

In [3]:
page = requests.get(url)

In [4]:
bs = soup(page.text, "html.parser")
#print(bs.prettify)

In [342]:
def extract_job_title_from_result(soup):
    '''Takes json result and find the job titles in each posting, return a list containing all queried job titles.
    '''
    job_post = []
    for div in bs.find_all(name="div", attrs={"class":"row"}):
        for a in div.find_all(name="a", attrs={"data-tn-element":"jobTitle"}):
            job_post.append(a["title"])
    return(job_post)
a = len(extract_job_title_from_result(bs))
a

26

In [341]:
def extract_company_from_result(soup):
    '''Takes json result and find the company titles in each posting, return a list containing all queried \n
    company titles.
    '''
    job_post = []
    for div in soup.find_all(name="div", attrs={"class":"row"}):
        company = soup.find_all(name="span", attrs={"class":"company"})
    if len(company) > 0:
        for b in company:
            job_post.append(b.text.strip())
    else:
        sec_try = div.find_all(name="span", attrs={"class":"turnstileLink"})
        for span in sec_try:
            job_post.append(span.text.strip())
    return(job_post)
    
a = len(extract_company_from_result(bs))
a

26

In [339]:
def extract_location_from_result(soup):
    '''Takes json result and find the location in each posting, return a list containing all queried locations.
    '''
    job_post = []
    # getting string in attribute
    spans = [item["data-rc-loc"] for item in soup.find_all() if "data-rc-loc" in item.attrs]
    for span in spans:
        job_post.append(span.strip())
    return(job_post)
a = len(extract_location_from_result(bs))
a

26

In [337]:
def extract_salary_from_result(soup):
    '''Takes json result and find the salaries in each posting, using string 'Nothing Found' when no salary \n
    infomration is present. Return a list containing all salaries.
    '''
    job_post = []
    for div in soup.find_all(name="div", attrs={"class":"row"}):
        try:
            s = div.find(name="span", attrs={"class":"salaryText"}).text
            job_post.append(s.strip())
        except:
                job_post.append("Nothing_found")
    return(job_post)
a = len(extract_salary_from_result(bs))
a

26

In [71]:
def extract_summary_from_result(soup):
    '''Takes json result and find the job summary in each posting. Return a list containing all s.
    '''
    job_post = []
    spans = soup.find_all(name="div", attrs={"class":"summary"})
    for span in spans:
        job_post.append(span.text.strip())
    return(job_post)
a = len(extract_summary_from_result(bs))
a

26

In [85]:
def extract_ratings_from_result(soup):
    '''Takes json result and find the rating in each posting. Return a list containing all ratings.
    '''
    job_post = []
    spans = soup.find_all(name="span", attrs={"class":"ratings"})
    #for span in spans:
        #job_post.append(span.text.strip())
    
    #? only 24/26 
    for span in spans:
        if "aria-label" in span.attrs:
                job_post.append(span['aria-label'])
        else:
            job_post.append("No_company_rating")
            
    print(spans)       
    return job_post

# close but no dice; element wrapping rating missing from campanies without rating
# no element to identify companies without rating, missed in find_all

In [2]:
# per csv
max_results = 1000

# originally wanted to query by city; have decided to gather location data as it comes ups
city_set = [""]

# initiate dataframe
columns = ["city", "job_title", "company_name", "location", "summary", "salary"]
df = pd.DataFrame(columns = columns)

# take user input of keyword in job search ex. data intern
query = input("Search Linkedin:")

In [3]:
for city in city_set:
    for start in range(0, max_results_per_city, 15):
        page = requests.get('http://www.indeed.ca/jobs?q=' + str(query) +'&l=' + str(city) + '&start=' + str(start))
        bs = soup(page.text, "html.parser")
        time.sleep(1)  #ensuring at least 1 second between page grabs
        for div in bs.find_all(name="div", attrs={"class":"row"}):
            job_post = []
            num = (len(df) + 1)
            job_post.append(city)
            for a in div.find_all(name="a", attrs={"data-tn-element":"jobTitle"}):
                job_post.append(a["title"])
            company = div.find_all(name="span", attrs={"class":"company"})
            if len(company) > 0:
                for b in company:
                    job_post.append(b.text.strip())
            else:
                sec_try = div.find_all(name="span", attrs={"class":"turnstileLink"})
                for span in sec_try:
                    job_post.append(span.text.strip())
            spans = [item["data-rc-loc"] for item in div.find_all() if "data-rc-loc" in item.attrs]
            for span in spans:
                job_post.append(span.strip())
            spans = div.find_all(name="div", attrs={"class":"summary"})
            for span in spans:
                job_post.append(span.text.strip())
            try:
                s = div.find(name="span", attrs={"class":"salaryText"}).text
                job_post.append(s.strip())
            except:
                job_post.append("Nothing_found")
            
            df.loc[num] = job_post

df.to_csv("test_scrape_no_city.csv")

In [335]:
job_post

['Toronto',
 'Intern - Technology, Risk & CISO (Winter 2020)',
 'Mackenzie Investments',
 'Toronto, ON',
 'Intern – Technology, Risk & CISO (IT Department). … Identify and analyze data to evaluate effectiveness of security controls.',
 'Nothing_found']

In [353]:
df

,city,job_title,company_name,location,summary,salary
1,Toronto,"Intern - Technology, Risk & CISO (Winter 2020)",Mackenzie Investments,"Toronto, ON","Intern – Technology, Risk & CISO (IT Departmen...",Nothing_found
2,Toronto,"Intern - Technology, Risk & CISO (Winter 2020)",Mackenzie Investments,"Toronto, ON","Intern – Technology, Risk & CISO (IT Departmen...",Nothing_found
3,Toronto,"Intern, Warehouse Operations Job",Metro Inc.,"Toronto, ON",The role of the Operations Intern is to provid...,Nothing_found
4,Toronto,Human Resources Internship (unpaid),Pillar Security,"Toronto, ON",The Human Resources Intern must possess degree...,Nothing_found
5,Toronto,IT Project Manager | Co-op Internship,GoIntegrations,"Scarborough, ON",GoIntregrations is looking for an Intern IT Pr...,Nothing_found
6,Toronto,HR Internship,Appliance Repair Team,"Concord, ON",We are looking for an HR intern to join our te...,Nothing_found
7,Toronto,Youth Program Event Coordinator,Basketball World Toronto,"Toronto, ON",JOB TITLE*– YOUTH PROGRAM EVENTS COORDINATOR. ...,$17.50 - $20.00 an hour
8,Toronto,Actuarial Analyst- Student Internship (Winter ...,Intact,"Toronto, ON",It’s more than a great career. … Participate i...,Nothing_found
9,Toronto,"Summer Intern, Data Science & Engineering",HSBC Bank Canada,"Toronto, ON",This could be building and changing predictive...,Nothing_found
10,Toronto,"Data Engineer, Omnia AI - Co-op/Intern Winter ...",Deloitte,"Toronto, ON","Fascinated with data structures, data models a...",Nothing_found


In [ ]:
<div class="location accessible-contrast-color-location">Toronto, ON</div>